In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import nan
import html_to_json
from unidecode import unidecode
import json
session = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}



In [85]:
def create_ticker(ticker,category,name,short_name,sector,document:str):
    data = {'ticker' : ticker,
            'category': category,
            'name': name,
            'short_name': short_name,
            'sector': sector,
            'document': document}   
    req = session.request('POST','http://localhost:3000/essentials',json=data)
    return print(req.status_code,req.text)

def get_content(ticker,typeof):
    req = session.request('GET',f'https://investidor10.com.br/{typeof}/{ticker}',headers=headers).text
    return req

def get_indicators_stock(ticker):
    output = {'ticker': ticker}
    site = BeautifulSoup(get_content(ticker,'acoes'), "html.parser")
    titulo = site.find_all(attrs={"class" : 'cell'})
    for i in range(len(titulo)):
        try:
            #Dividend Yield duplica o ticker
            if 'DIVIDEND YIELD' in html_to_json.convert(str(titulo[i]))['div'][0]['span'][0]['_value']:
                if html_to_json.convert(str(titulo[i]))['div'][0]['div'][0]['span'][0]['_value'] != '-':
                    output['DIVIDEND YIELD'] = html_to_json.convert(str(titulo[i]))['div'][0]['div'][0]['span'][0]['_value']
                else:
                    output['DIVIDEND YIELD'] = '0'
            #{indicator name: indicator value}    
            else:
                if html_to_json.convert(str(titulo[i]))['div'][0]['div'][0]['span'][0]['_value'] != '-':
                    
                    output[html_to_json.convert(str(titulo[i]))['div'][0]['span'][0]['_value']] = html_to_json.convert(str(titulo[i]))['div'][0]['div'][0]['span'][0]['_value']
                else:
                    output[html_to_json.convert(str(titulo[i]))['div'][0]['span'][0]['_value']] = '0'
        except:
            pass
    return output

def get_results_all_stocks():
    current_overview = pd.DataFrame(session.request("GET","http://localhost:3000/position_overview").json())
    current_overview = current_overview[current_overview.current_position > 0]
    active_stocks = current_overview[current_overview.ess_category == 'Ação']
    active_stocks = active_stocks['ess_ticker'].to_list()
    list_output = []
    for ticker in active_stocks:
        list_output.append(get_indicators_stock(ticker))
    return list_output

def get_current_price(ticker):
    output = {}
    google = session.request('GET',f'https://www.google.com/finance/quote/{ticker}:BVMF').text
    site = BeautifulSoup(google, "html.parser")
    titulo = site.find_all(attrs={"class" : 'YMlKec fxKbKc'})
    current_price = html_to_json.convert(str(titulo))['div'][0]['_value']
    output['ticker'] = ticker
    output['price'] = current_price
    return output

def data_format(list_output):
    results = list_output
    current_overview = pd.DataFrame(results)
    for i in current_overview.columns:
        try:
            current_overview[i] = current_overview[i].apply(lambda x: str(float(x.replace('%','').replace(',','.'))/100) if '%' in str(x) else str(x).replace('%','').replace(',','.'))
        except:
            pass
    current_overview.rename(columns={ 
    'P/L': 'pl',
    'P/RECEITA (PSR)': 'psr',
    'P/VP': 'pvp',
    'DIVIDEND YIELD': 'dividend_yield',
    'PAYOUT': 'payout',
    'MARGEM LÍQUIDA': 'margem_liquida',
    'MARGEM BRUTA': 'margem_bruta',
    'MARGEM EBIT': 'margem_ebit',
    'MARGEM EBITDA': 'margem_ebitda',
    'EV/EBITDA': 'evebitda',
    'EV/EBIT': 'evebit',
    'P/EBITDA': 'pebitda',
    'P/EBIT': 'pebit',
    'P/ATIVO': 'pativo',
    'P/CAP.GIRO': 'pcap_giro',
    'P/ATIVO CIRC LIQ': 'pativo_circ_liq',
    'VPA': 'vpa',
    'LPA': 'lpa',
    'GIRO ATIVOS': 'giro_ativos',
    'ROE': 'roe',
    'ROIC': 'roic',
    'ROA': 'roa',
    'DÍVIDA LÍQUIDA / PATRIMÔNIO': 'divida_liquida_patrimonio',
    'DÍVIDA LÍQUIDA / EBITDA': 'divida_liquida_ebitda',
    'DÍVIDA LÍQUIDA / EBIT': 'divida_liquida_ebit',
    'DÍVIDA BRUTA / PATRIMÔNIO': 'divida_bruta_patrimonio',
    'PATRIMÔNIO / ATIVOS': 'patrimonio_ativos',
    'PASSIVOS / ATIVOS': 'passivos_ativos',
    'LIQUIDEZ CORRENTE': 'liquidez_corrente',
    'CAGR RECEITAS 5 ANOS': 'cagr_receitas_5_anos',
    'CAGR LUCROS 5 ANOS': 'cagr_lucros_5_anos'}
                            ,inplace=True)
    return current_overview.to_dict(orient='records')

def update_indicators():
    results = get_results_all_stocks()
    pack = data_format(results)
    req = requests.request('POST','http://localhost:3000/performance_results',data=json.dumps(pack))
    return print(req.status_code,'Atualizado')



In [61]:
def get_results_all_stocks():
    current_overview = pd.DataFrame(session.request("GET","http://localhost:3000/position_overview").json())
    current_overview = current_overview[current_overview.current_position > 0]
    active_stocks = current_overview[current_overview.ess_category == 'Ação']
    active_stocks = active_stocks['ess_ticker'].to_list()
    list_output = []
    for ticker in active_stocks:
        
        list_output.append(get_indicators_stock(ticker))
    return list_output

In [76]:
current_overview = pd.DataFrame(requests.request("GET","http://localhost:3000/position_overview").json())
current_overview = current_overview[current_overview.current_position > 0]
active_stocks = current_overview[current_overview.ess_category == 'Ação']
active_stocks = active_stocks[['ess_ticker','ess_id']]
active_stocks

,ess_ticker,ess_id
1,ALOS3,5d10c2ca-aad1-4df8-b12e-a363b500f488
2,MEAL3,71ab38e1-93f0-4ff5-8c98-78d84ffd0c00
3,GOAU3,3093c429-849d-4ddd-a398-1676d6c9aace
5,SUZB3,d1d6fe75-75d8-42c0-9a18-005a524d9c6f
9,ITSA4,a1c907bf-edf7-48e5-9d9d-f75a56e7307e
11,RAPT4,867ab30c-4356-43b6-9bc0-8a68334d0190
12,ENGI4,e58ecf01-86d8-45a7-a179-35f4494e9fda
14,ABCB4,3bd0b7b8-0e3c-4dfa-b960-3fb4b5c33aab
15,TAEE11,c0b177cc-a0d0-4e26-8082-0a77442e010b
16,SLCE3,218cc579-6ed4-40a7-ac01-a3728f5fba5c


In [62]:

current_overview = pd.DataFrame(results)


'npm' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [81]:
pd.DataFrame(requests.request('GET','http://localhost:3000/performance_results').json())

,perfomance_id,created_at,ticker,pl,psr,pvp,dividend_yield,payout,margem_liquida,margem_bruta,...,roa,divida_liquida_patrimonio,divida_liquida_ebitda,divida_liquida_ebit,divida_bruta_patrimonio,patrimonio_ativos,passivos_ativos,liquidez_corrente,cagr_receitas_5_anos,cagr_lucros_5_anos
0,84b9bb15-ff45-4267-892a-d641a2628349,2024-07-20T02:00:31.789361,ALOS3,23.41,4.31,0.85,0.0516,1.4883,0.1839,0.7106,...,0.0228,-0.17,-1.45,-2.41,0,0.51,0.45,1.29,0.2977,0.615
1,10346ee0-c898-49a8-83cd-d9984e4dc833,2024-07-20T02:00:31.789361,MEAL3,-7.84,0.18,0.41,0.0000,0.0000,-0.0234,0.3455,...,-0.0198,0.32,1.67,4.02,0.53,0.38,0.62,1.04,0.0702,NaN
2,59ac31f4-7a56-4882-be47-cc8977ebb511,2024-07-20T02:00:31.789361,GOAU3,5.33,0.17,0.65,0.1283,0.6865,0.0325,0.1528,...,0.0828,0.27,0.4,0.54,0.62,0.23,0.32,2.82,0.1082,0.3986
3,93f96985-5b0e-4ccd-98a1-7665787707be,2024-07-20T02:00:31.789361,SUZB3,7.71,1.84,1.57,0.0216,0.1673,0.2390,0.3461,...,0.0633,1.35,3.33,5.74,1.77,0.31,0.69,2.89,0.0784,NaN
4,ca68ab25-f043-48d8-984c-347bf9f5d50c,2024-07-20T02:00:31.789361,ITSA4,7.57,14.07,1.33,0.0836,0.7284,1.8592,0.2975,...,0.1398,0.05,0.24,0.26,0.15,0.77,0.19,2.71,0.0872,0.0658
5,abbe607f-eca0-44b7-896c-e22f7125949c,2024-07-20T02:00:31.789361,RAPT4,11.32,0.36,1.34,0.0490,0.5612,0.0317,0.2551,...,0.0470,1.05,2.05,2.58,2.09,0.21,0.70,1.89,0.1616,0.1619
6,157f8eca-3e45-4386-8488-c10a182e36de,2024-07-20T02:00:31.789361,ENGI4,7.60,0.61,1.19,0.0424,0.3249,0.0800,0.2852,...,0.0453,1.52,2.8,3.49,2.12,0.21,0.73,1.28,0.0853,0.4352
7,f1097501-5335-4734-a537-a5cbb2fed234,2024-07-20T02:00:31.789361,ABCB4,6.25,0.89,0.90,0.0676,0.6533,0.1423,0.2243,...,0.0147,NaN,NaN,NaN,NaN,0.10,0.90,1.67,0.1852,0.1085
8,d8a0c880-5822-4a71-8394-33bf7c696a7a,2024-07-20T02:00:31.789361,TAEE11,8.62,3.43,1.65,0.1082,0.9647,0.3985,0.6314,...,0.0690,1.23,3.73,3.75,1.38,0.36,0.64,1.26,0.1363,0.0622
9,b34e4b6a-29c9-47ce-b9c4-11424fa76677,2024-07-20T02:00:31.789361,SLCE3,14.80,1.07,1.74,0.0458,0.6764,0.0721,0.2610,...,0.0384,0.63,1.84,2.13,1.01,0.32,0.66,1.67,0.2147,0.1344


In [80]:
update_indicators()
pd.DataFrame(requests.request('GET','http://localhost:3000/performance_results').json())

201 Atualizado


,perfomance_id,created_at,ticker,pl,psr,pvp,dividend_yield,payout,margem_liquida,margem_bruta,...,roa,divida_liquida_patrimonio,divida_liquida_ebitda,divida_liquida_ebit,divida_bruta_patrimonio,patrimonio_ativos,passivos_ativos,liquidez_corrente,cagr_receitas_5_anos,cagr_lucros_5_anos
0,84b9bb15-ff45-4267-892a-d641a2628349,2024-07-20T02:00:31.789361,ALOS3,23.41,4.31,0.85,0.0516,1.4883,0.1839,0.7106,...,0.0228,-0.17,-1.45,-2.41,0,0.51,0.45,1.29,0.2977,0.615
1,10346ee0-c898-49a8-83cd-d9984e4dc833,2024-07-20T02:00:31.789361,MEAL3,-7.84,0.18,0.41,0.0000,0.0000,-0.0234,0.3455,...,-0.0198,0.32,1.67,4.02,0.53,0.38,0.62,1.04,0.0702,NaN
2,59ac31f4-7a56-4882-be47-cc8977ebb511,2024-07-20T02:00:31.789361,GOAU3,5.33,0.17,0.65,0.1283,0.6865,0.0325,0.1528,...,0.0828,0.27,0.4,0.54,0.62,0.23,0.32,2.82,0.1082,0.3986
3,93f96985-5b0e-4ccd-98a1-7665787707be,2024-07-20T02:00:31.789361,SUZB3,7.71,1.84,1.57,0.0216,0.1673,0.2390,0.3461,...,0.0633,1.35,3.33,5.74,1.77,0.31,0.69,2.89,0.0784,NaN
4,ca68ab25-f043-48d8-984c-347bf9f5d50c,2024-07-20T02:00:31.789361,ITSA4,7.57,14.07,1.33,0.0836,0.7284,1.8592,0.2975,...,0.1398,0.05,0.24,0.26,0.15,0.77,0.19,2.71,0.0872,0.0658
5,abbe607f-eca0-44b7-896c-e22f7125949c,2024-07-20T02:00:31.789361,RAPT4,11.32,0.36,1.34,0.0490,0.5612,0.0317,0.2551,...,0.0470,1.05,2.05,2.58,2.09,0.21,0.70,1.89,0.1616,0.1619
6,157f8eca-3e45-4386-8488-c10a182e36de,2024-07-20T02:00:31.789361,ENGI4,7.60,0.61,1.19,0.0424,0.3249,0.0800,0.2852,...,0.0453,1.52,2.8,3.49,2.12,0.21,0.73,1.28,0.0853,0.4352
7,f1097501-5335-4734-a537-a5cbb2fed234,2024-07-20T02:00:31.789361,ABCB4,6.25,0.89,0.90,0.0676,0.6533,0.1423,0.2243,...,0.0147,NaN,NaN,NaN,NaN,0.10,0.90,1.67,0.1852,0.1085
8,d8a0c880-5822-4a71-8394-33bf7c696a7a,2024-07-20T02:00:31.789361,TAEE11,8.62,3.43,1.65,0.1082,0.9647,0.3985,0.6314,...,0.0690,1.23,3.73,3.75,1.38,0.36,0.64,1.26,0.1363,0.0622
9,b34e4b6a-29c9-47ce-b9c4-11424fa76677,2024-07-20T02:00:31.789361,SLCE3,14.80,1.07,1.74,0.0458,0.6764,0.0721,0.2610,...,0.0384,0.63,1.84,2.13,1.01,0.32,0.66,1.67,0.2147,0.1344


In [74]:
requests.request('DELETE','http://localhost:3000/performance_results')

<Response [204]>

201 
